In [ ]:
import math
import powerlaw
import statistics

import networkx as nx
import matplotlib.pyplot as plt

from collections import Counter

In [ ]:
# Load the chosen graph
graph_name = "net_ga_all"
graph_path = "../graphs/{}.gml".format(graph_name)
net = nx.read_gml(graph_path)

In [ ]:
for edge in net.edges(data=True):
    edge[2]['reciprocal'] = 1 / edge[2]['weight']

### Basic Characteristics

In [ ]:
print(f'Total countries: {net.number_of_nodes()}')

In [ ]:
# Density
print(f'Density: {nx.density(net)}')

In [ ]:
# Average Distance and Diameter
print(f'Average distance: {nx.average_shortest_path_length(net)}')
print(f'Diameter: {nx.diameter(net)}')

In [ ]:
print(f'Is connected: {nx.is_connected(net)}')

In [ ]:
print(f'Average clustering: {nx.average_clustering(net, weight="weight")}')
print(f'Global clustering: {nx.transitivity(net)}')

In [ ]:
def plot_clustering(net, weight = None, logScale = False):
    clustering = nx.clustering(net, weight=weight)
    for key, value in clustering.items():
        clustering[key] = math.floor(value * 40) / 40

    clust_counts = Counter(clustering.values())
    print(clust_counts)
    x, y = zip(*clust_counts.items())

    plt.ylabel('frequency')
    plt.xlabel('weighted clustering') if weight != None else plt.xlabel('clustering')

    plt.bar(clust_counts.keys(), clust_counts.values(), 0.025, align='edge', color='darkred', log=logScale, edgecolor="black")

In [ ]:
plot_clustering(net, weight = "weight", logScale = False)

In [ ]:
# Create equivalent Erdos Renyi and ScaleFree networks
n = net.number_of_nodes()
m = net.number_of_edges()
p = ( 2*float(m) ) / ( n* (n-1) )

netER = nx.erdos_renyi_graph(n, p)
netSFMulti = nx.scale_free_graph(n)

netSF = nx.DiGraph()
for u,v in netSFMulti.edges():
    if netSF.has_edge(u,v):
        netSF[u][v]['weight'] += 1
    else:
        netSF.add_edge(u, v, weight=1)

In [ ]:
print(f'ER Average clustering: {nx.average_clustering(netER)}')
print(f'SF Average clustering: {nx.average_clustering(netSF)}')

In [ ]:
print(f"Netežinski koeficijent asortativnosti: {nx.degree_assortativity_coefficient(net)}")
print(f"Težinski koeficijent asortativnosti: {nx.degree_assortativity_coefficient(net, weight='weight')}")

In [ ]:
netDegrees = nx.degree(net, weight='weight')
richClubNet = nx.subgraph(net, [x for x in net.nodes() if netDegrees[x] > 14000])

In [ ]:
print(f'Rich Club Density: {nx.density(richClubNet)}')

In [ ]:
# Plot degree distrubution
def plot_deg_frequency(net, weight = None, xscale = "log", yscale = "log"):
    degrees = dict(net.degree(weight=weight))
    for key, value in degrees.items():
        degrees[key] = 5 * round(value/5)

    deg_counts = Counter(degrees.values())
    print(deg_counts)
    x, y = zip(*deg_counts.items())

    plt.figure(1)
    
    plt.xlabel('weighted degree') if weight != None else plt.xlabel('degree')
    plt.xscale(xscale)
    plt.xlim(min(x), max(x))

    plt.ylabel('frequency')
    plt.yscale(yscale)
    plt.ylim(1, max(y))

    plt.scatter(x, y, marker='.')
    plt.show()

In [ ]:
# Show non-weighted degree distrubution
plot_deg_frequency(net, xscale='linear', yscale='linear')

In [ ]:
# Show weighted degree distrubution
plot_deg_frequency(net, weight='weight', xscale='linear', yscale='linear')

### Least and most friendly country duos

In [ ]:
def plot_agreement(net, xscale = "log", yscale = "log"):
    agreements = [ edge[2]['agreement'] for edge in net.edges(data=True) ]
    agreements = list(map(lambda x: math.floor(x * 40) / 40, agreements))
    
    print(f'Median agreement: {statistics.median(agreements)}')
    print(f'Average agreement: {statistics.fmean(agreements)}')
    
    agreement_counts = Counter(agreements)
    print(agreement_counts)
    x, y = zip(*agreement_counts.items())

    plt.figure(1)
    
    plt.xlabel('agreement')
    plt.xscale(xscale)
    plt.xlim(0, max(x))

    plt.ylabel('frequency')
    plt.yscale(yscale)
    plt.ylim(1, max(y))

    plt.scatter(x, y, marker='.')
    plt.show()

In [ ]:
# Show the distribution of 'agreement' values
plot_agreement(net, xscale='linear', yscale='linear')

In [ ]:
def friendly_edge_to_string(edge, country_buffer_1, country_buffer_2):
    country1 = edge[0].ljust(country_buffer_1)
    country2 = edge[1].ljust(country_buffer_2)
    
    total = str(edge[2]['total']).ljust(5)
    points = str(edge[2]['points']).rjust(6)
    agreement = round(edge[2]['agreement'] * 100, 1)
    
    return '{}, {} - {}/{} ({}%)'.format(country1, country2, points, total, agreement)

def edge_country_1_length(edge):
    return len(edge[0])

def edge_country_2_length(edge):
    return len(edge[1])

def edge_country_2_length(edge):
    return len(edge[1])

def friendly_edge_print(edges):
    max_cnt_1_length = max(map(edge_country_1_length, edges))
    max_cnt_2_length = max(map(edge_country_2_length, edges))
    for edge_str in map(lambda e: friendly_edge_to_string(e, max_cnt_1_length, max_cnt_2_length), edges):
        print(edge_str)

In [ ]:
# Sort edges by agreement
def has_significant_total(edge):
    return edge[2]['total'] >= 0

sorted_edges = sorted(net.edges(data=True), key=lambda edge: edge[2]['agreement'])
sorted_edges =  list(__builtin__.filter(has_significant_total, sorted_edges))

In [ ]:
# Show countries with the least agreement
friendly_edge_print(sorted_edges[:5])

In [ ]:
# Show countries with the most agreement
friendly_edge_print(sorted_edges[-5:])

### Centrality

In [ ]:
def print_centrality(data, high_count=5, low_count=5):
    vals = sorted(data.items(), key=lambda x: x[1], reverse=True)
    vals = list(map(lambda val: (val[0], round(val[1], 1)), vals))
    
    print('Higest values:')
    for val in vals[:high_count]:
        print(f'{val[0]}: {val[1]}');
        
    print('\nLowest values:')
    for val in vals[-low_count:]:
        print(f'{val[0]}: {val[1]}');

In [ ]:
# Print degree centrality
print_centrality(dict(net.degree(weight='agreement')), 5, 5)

In [ ]:
# Print closeness centrality
print_centrality(dict(nx.closeness_centrality(net)), 5, 5)

In [ ]:
# Print closeness centrality
print_centrality(dict(nx.betweenness_centrality(net, weight='reciprocal', normalized=False)), 6, 5)

In [ ]:
print_centrality(nx.eigenvector_centrality(net, max_iter=1000, weight='reciprocal'), 10, 10)

### Additional TODO list

Power-Law\
Core and Periphery\
Communues